In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import numpy as np
import re
import contractions
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import spacy

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/NLP/combined_file.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,title,text,score,upvote_ratio,upvotes,Unnamed: 5,Unnamed: 6
0,Looking for married Muslim men who have hijabi...,Don’t message me if you can’t live verify. Too...,1,1,1,NaN,NaN
1,Share your istikhara success stories. I need s...,Salaam everyone. I’m a F currently going throu...,1,1,1,NaN,NaN
2,Fate?,"In the Qur'an, I saw verses in these cases tha...",1,1,1,NaN,NaN
3,Good Thrift shop find. Highly reccomend,Holocaust book about family of Jewish Hungaria...,1,1,1,NaN,NaN
4,Wearing my kippah with tattoos,Shalom friends!\n\nI’m a Baal teshuva with man...,1,1,1,NaN,NaN


In [ ]:
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)

In [ ]:
df = df.drop(columns=['Unnamed: 5', 'Unnamed: 6'])

In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
url_pattern = r'http[s]?://[^\s]+'
urls_found = df[df['text'].str.contains(url_pattern, regex=True)]

print("Sentences with URLs:")
print(urls_found[['text']])

Sentences with URLs:
                                                    text
9      i found an old bbc documentary show titled "co...
26     /r/worldnews\n   \n https://www.reuters.com/wo...
31           https://www.youtube.com/watch?v=v3ga7skxino
50     this is an automated weekly thread\n\nfeel fre...
61     /r/worldnews\n   \n https://www.forbes.com/sit...
...                                                  ...
92653  israeli authorities have declined to release a...
92656  /r/worldnews\n   \n https://www.businessinside...
92657  it is important for those new to the palestini...
92662  [https://bloggingtheology.net/2016/05/06/chris...
92669  my previous post was removed by a mod because ...

[17485 rows x 1 columns]


In [ ]:
df['text'] = df['text'].str.replace(url_pattern, '', regex=True)

In [ ]:
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
emails_found = df[df['text'].str.contains(email_pattern, regex=True)]

print("\nSentences with email addresses:")
print(emails_found[['text']])


Sentences with email addresses:
                                                    text
7798   i'm running a speaker event on monday evening,...
14458   hi i'm a disabled  dad if 2, london uk\nmy wi...
19960  i am excited that laasok - the new liberal bei...
22489  i need a couple of volunteers for tomorrow.  s...
25704  salamalaikum to everyone. i want to help out s...
25962  hi all. i'm a reform rabbi, and founder of laa...
25966  hello! we are a group of undergraduates lookin...
26537  hello, my name is veronica, and i am a doctora...
26540  hello, my name is veronica, and i am a doctora...
26848  \n\nanasya's donation link: anastasiyaparaskev...
40437  this is their official [post on fb:](\n\ndear ...
41226  on 12/10/2023 on lbc at around 7:00pm-8:00pm t...
42619  binance, the world's largest cryptocurrency ex...
43429  \n\tplease use the sharing tools found via the...
48964    \nplease use the sharing tools found via the...
55899  reform judaism is often referred to as "non-ha..

In [ ]:
df['text'] = df['text'].str.replace(email_pattern, '', regex=True)

In [ ]:
hashtag_pattern = r'#[\w]+'
hashtags_found = df[df['text'].str.contains(hashtag_pattern, regex=True)]

print("\nSentences with hashtags:")
print(hashtags_found[['text']])


Sentences with hashtags:
                                                    text
5      slightly embarrassing question but also someth...
6      i found this subreddit not too long ago but up...
50     this is an automated weekly thread\n\nfeel fre...
93     \* you live in the west and are casually walki...
96                                                #name?
...                                                  ...
92589                       &#x200b;\n\n\n\n&#x200b;\n\n
92595                       &#x200b;\n\n\n\n&#x200b;\n\n
92618  hi all,\n\ni wrote my congressman today (2nd t...
92653  israeli authorities have declined to release a...
92657  it is important for those new to the palestini...

[4728 rows x 1 columns]


In [ ]:
df['text'] = df['text'].str.replace(hashtag_pattern, '', regex=True)

In [ ]:
df['text'] = df['text'].str.strip().str.replace('\s+', ' ', regex=True)

In [ ]:
df['text'] = df['text'].apply(contractions.fix)

In [ ]:
def remove_special_characters(text):
    return re.sub(r'[^A-Za-z0-9\s]', '', text)

df['text'] = df['text'].apply(remove_special_characters)

In [ ]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

df['text'] = df['text'].apply(remove_numbers)

In [ ]:
df['text'] = df['text'].str.strip().str.replace('\s+', ' ', regex=True)

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def preprocess(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    pos_tags = pos_tag(filtered_tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

In [ ]:
df['text'] = df['text'].apply(preprocess)

In [ ]:
entity = ["hamas", "israel"]

In [ ]:
def extract_entity_context(text, entity_list):
    doc = nlp(text)
    sentences_with_entities = []

    for sent in doc.sents:
        if any(ent.text in entity_list for ent in sent.ents):
            sentences_with_entities.append(sent.text)

    return sentences_with_entities

In [ ]:
df['entity_context'] = df['text'].apply(lambda x: extract_entity_context(x, entity))

In [ ]:
df = df[df['entity_context'].str.len() > 0]


In [ ]:
df['entity_context']

,entity_context
84,[case germany right author claim jewish year p...
91,[melachim ii ii king chapter ahaziah twentytwo...
102,[hello interest know jewish israeli think arab...
105,[may move israel near future take job offer je...
121,[expect mail israeli university outside israel...
...,...
92650,[see uptick amount comment accuse people suppo...
92653,[israeli authority decline release ahmad manas...
92657,[important new palestinian topic understand pa...
92661,[past two month leave frustrated upset native ...
